In [1]:
import mne
import pandas as pd
import numpy as np
from pathlib import Path

# --- 1. Setup Paths ---
# Current folder (where this script is running)
scripts_folder = Path.cwd()

# Go up two levels to find the root, then down to data
root_folder = scripts_folder.parent.parent
data_path = root_folder / 'data' / 'ds003838' / 'sub-032'
data_folder = data_path / 'ecg'

print(f"📂 Reading data from: {data_folder}")
print(f"💾 Saving output to:  {scripts_folder}")

subject_id = 'sub-032'
task = 'rest'

# --- 2. Load Signal (.set) ---
set_filename = f"{subject_id}_task-{task}_ecg.set"
set_path = data_folder / set_filename

if set_path.exists():
    print("⏳ Loading .set file (this might take a moment)...")
    try:
        # Load the binary file with MNE
        raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
        
        # Convert to DataFrame
        # This creates columns: 'time', 'ECG', etc.
        df = raw.to_data_frame()
        
        # --- 3. Load & Merge Events (.tsv) ---
        tsv_filename = f"{subject_id}_task-{task}_events.tsv"
        tsv_path = data_folder / tsv_filename
        
        if tsv_path.exists():
            print("⏳ Merging events...")
            events = pd.read_csv(tsv_path, sep='\t')
            
            # Initialize a column for events
            df['event_label'] = 'baseline'
            
            # Stamp events onto the time series
            for _, row in events.iterrows():
                # Find rows in the signal that fall within the event duration
                mask = (df['time'] >= row['onset']) & (df['time'] <= (row['onset'] + row['duration']))
                
                # Get the label (column name is usually 'trial_type' or 'value')
                label = row.get('trial_type', row.get('value', 'event'))
                
                if row['duration'] == 0:
                    # For instant markers, find the nearest single sample
                    idx = (df['time'] - row['onset']).abs().idxmin()
                    df.at[idx, 'event_label'] = label
                else:
                    df.loc[mask, 'event_label'] = label
        else:
            print(f"⚠️  Events file not found: {tsv_filename}")

        # --- 4. Save as Parquet in Scripts Folder ---
        parquet_filename = f"{subject_id}_task-{task}_ecg.parquet"
        output_path = scripts_folder / parquet_filename
        
        df.to_parquet(output_path, index=False)
        
        print(f"✅ Success! Parquet saved at: {output_path}")
        print(f"   DataFrame shape: {df.shape}")
        print(f"   Columns: {list(df.columns)}")
        
    except Exception as e:
        print(f"❌ Error processing file: {e}")
else:
    print(f"⚠️  Signal file not found: {set_filename}")

📂 Reading data from: /home/martin/RESEARCH/thesis/code/brain_heart_psv_sdg/data/ds003838/sub-032/ecg
💾 Saving output to:  /home/martin/RESEARCH/thesis/code/brain_heart_psv_sdg/code/digitspan
⏳ Loading .set file (this might take a moment)...


/home/martin/miniforge3/envs/brain-heart/lib/python3.11/site-packages/pymatreader/utils.py:179: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(


⏳ Merging events...
✅ Success! Parquet saved at: /home/martin/RESEARCH/thesis/code/brain_heart_psv_sdg/code/digitspan/sub-032_task-rest_ecg.parquet
   DataFrame shape: (227081, 4)
   Columns: ['time', 'PPG', 'ECG', 'event_label']


/tmp/ipykernel_168965/2510212186.py:29: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
/tmp/ipykernel_168965/2510212186.py:29: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
